In [12]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트 
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 rand, when, pandas_udf, PandasUDFType
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 BooleanType
import pandas as pd

In [2]:
# 생성 a new SparkSession
spark = (SparkSession
         .builder
         .appName("broadcast-variables")  # 애플리케이션 이름 설정
         .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
         .config("spark.executor.memory", "512m")  # Spark 설정 옵션
         .getOrCreate()  # SparkSession 생성 또는 기존 세션 반환)

# Set log level to ERROR
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/21 12:33:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# 생성 some sample data frames
# A large data frame with 1 million rows
large_df = (spark.range(0, 1000000)
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"salary", 100*(rand() * 100).cast("int"))
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"gender", when((rand() * 2).cast("int") == 0, "M").otherwise("F"))
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"country_code", 
                        when(  # 조건문 시작(rand() * 4).cast("int") == 0, "US")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 1, "CN")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 2, "IN")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 3, "BR")))
large_df.show(5)

+---+------+------+------------+
| id|salary|gender|country_code|
+---+------+------+------------+
|  0|  8000|     M|          US|
|  1|  3500|     F|        null|
|  2|  9700|     F|        null|
|  3|  4800|     F|        null|
|  4|  9100|     F|        null|
+---+------+------+------------+
only showing top 5 rows



In [8]:
# Define lookup table
lookup = {"US": "United States", "CN": "China", "IN": "India", "BR": "Brazil", "RU": "Russia"}

# 생성 broadcast variable
broadcast_lookup = spark.sparkContext.broadcast(lookup)

In [9]:
@pandas_udf(  # 사용자 정의 함수 생성'string', PandasUDFType.SCALAR)
def country_convert(s):
    return s.map(broadcast_lookup.value)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [10]:
large_df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"country_name", country_convert(large_df.country_code)).show(5)

+---+------+------+------------+-------------+
| id|salary|gender|country_code| country_name|
+---+------+------+------------+-------------+
|  0|  8000|     M|          US|United States|
|  1|  3500|     F|        null|         null|
|  2|  9700|     F|        null|         null|
|  3|  4800|     F|        null|         null|
|  4|  9100|     F|        null|         null|
+---+------+------+------------+-------------+
only showing top 5 rows



In [13]:
@pandas_udf(  # 사용자 정의 함수 생성BooleanType(), PandasUDFType.SCALAR)
def filter_unknown_country(s):
    return s.isin(broadcast_lookup.value)

In [14]:
large_df.filter(  # 데이터 필터링filter_unknown_country(large_df.country_code)).show(5)

+---+------+------+------------+
| id|salary|gender|country_code|
+---+------+------+------------+
|  0|  8000|     M|          US|
|  6|  3400|     F|          US|
|  7|  8400|     M|          CN|
|  8|  1100|     F|          US|
|  9|  2900|     M|          CN|
+---+------+------+------------+
only showing top 5 rows



In [15]:
spark.stop()  # Spark 세션 종료 - 리소스 정리